In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import httpx
from brokit.primitives.lm import LM, ModelType, ModelResponse, Usage, Message
from typing import List, Optional

class Llama(LM):
    def __init__(self, model_name: str, base_url:str = "http://localhost:11434", temperature:float=0.0, top_p:float=1.0, seed:int=55, **kwargs):
        super().__init__(model_name=model_name, model_type=ModelType.CHAT)
        self.base_url = base_url
        self.client = httpx.Client(timeout=60.0)  # Reusable client)
        self.model_params = {
            "temperature": temperature,
            "top_p": top_p,
            "seed": seed,
            **kwargs
        }

    def request(self, prompt:Optional[str]=None, messages:Optional[List[Message]]=None, **kwargs) -> dict:
        url = f"{self.base_url}/api/chat"
        params = {**self.model_params, **kwargs}
        if messages is not None:
            _messages = [msg.to_dict() if isinstance(msg, Message) else msg for msg in messages]
        else:
            _messages = [{"role": "user", "content": prompt}]
        response = self.client.post(
            url,
            json={
                "model": self.model_name,
                "messages": _messages,
                "stream": False,
                "options": {**params},
            }
        )                
        return response.json()

    def parse_response(self, original_response: dict) -> ModelResponse:
        message = original_response["message"]
        input_tokens = original_response.get("prompt_eval_count", 0)
        output_tokens = original_response.get("eval_count", 0)
        return ModelResponse(
            model_name=self.model_name,
            model_type=self.model_type,
            response=message["content"],
            usage=Usage(input_tokens=input_tokens, output_tokens=output_tokens),
            metadata=None
        )

In [3]:
from brokit.primitives.prompt import Prompt, InputField, OutputField

class QA(Prompt):
    """Answer a question based on the provided context."""
    question:str = InputField(description="The question to be answered.")
    dice:int = OutputField(description="roll the dice, its outcome ranging from 1 to 6")
    logic:bool = OutputField(description="if the dice's outcome is greater than or equal to 5, logic=True, otherwise False")
    answer:str = OutputField(description="If logic=True, answer the question in sarcastic tone, else smooth-tone.")

In [4]:
from brokit.primitives.predictor import Predictor

# lm = Llama(model_name="gemma3:12b")
lm = Llama(model_name="gemma3:12b")
predictor = Predictor(prompt=QA, lm=lm)

In [5]:
prediction = predictor(question="How ya been?")
prediction

Prediction(
    dice=3,
    logic=False,
    answer="I've been quite well, thank you for asking! Just doing my usual AI things. How about yourself?"
)

In [6]:
prediction.dice, prediction.logic, prediction.answer

(3,
 False,
 "I've been quite well, thank you for asking! Just doing my usual AI things. How about yourself?")

In [7]:
lm.history[-1]

ModelResponse(model_name='gemma3:12b', model_type=<ModelType.CHAT: 'chat'>, response="<||dice||>\n3\n\n<||logic||>\nFalse\n\n<||answer||>\nI've been quite well, thank you for asking! Just doing my usual AI things. How about yourself?\n\n<||completed||>", usage=Usage(input_tokens=259, output_tokens=52), response_ms=3081.1028000025544, cached=False, metadata=None, request=[Message(role='system', content="Your input fields are:\n1. question (<class 'str'>): The question to be answered.\nYour output fields are:\n1. dice (<class 'int'>): roll the dice, its outcome ranging from 1 to 6\n2. logic (<class 'bool'>): if the dice's outcome is greater than or equal to 5, logic=True, otherwise False\n3. answer (<class 'str'>): If logic=True, answer the question in sarcastic tone, else smooth-tone.\n\nAll interactions will be structured in the following way, with the appropriate values filled in.\n\n<||question||>\n{question}\n\n<||dice||>\n{dice}\n\n<||logic||>\n{logic}\n\n<||answer||>\n{answer}\n\n

In [8]:
lm.history[-1].response

"<||dice||>\n3\n\n<||logic||>\nFalse\n\n<||answer||>\nI've been quite well, thank you for asking! Just doing my usual AI things. How about yourself?\n\n<||completed||>"

In [9]:
lm.history[-1].request

[Message(role='system', content="Your input fields are:\n1. question (<class 'str'>): The question to be answered.\nYour output fields are:\n1. dice (<class 'int'>): roll the dice, its outcome ranging from 1 to 6\n2. logic (<class 'bool'>): if the dice's outcome is greater than or equal to 5, logic=True, otherwise False\n3. answer (<class 'str'>): If logic=True, answer the question in sarcastic tone, else smooth-tone.\n\nAll interactions will be structured in the following way, with the appropriate values filled in.\n\n<||question||>\n{question}\n\n<||dice||>\n{dice}\n\n<||logic||>\n{logic}\n\n<||answer||>\n{answer}\n\n<||completed||>\nIn adhering to this structure, your objective is: \nAnswer a question based on the provided context.", images=None),
 Message(role='user', content='<||question||>\nHow ya been?\n\nRespond with the corresponding output fields, starting with the field: `<||dice||>`, `<||logic||>`, `<||answer||>` and then ending with the marker for `<||completed||>`.', imag

In [10]:
class ImageDescriptor(Prompt):
    """Describe images"""
    description:str = OutputField(description="description of images")


In [11]:
ImageDescriptor.input_fields

{}

In [12]:
ImageDescriptor.output_fields

{'description': FieldInfo(name='description', description='description of images', type=<class 'str'>, is_input=False)}

In [13]:
ImageDescriptor.instructions

'Describe images'

In [14]:
ip = Predictor(ImageDescriptor, lm=lm)

In [15]:
from brokit.primitives.prompt import Image
doraemon1 = Image("https://aithailand.co.th/wp-content/uploads/2024/09/Doraemon-th_1630640671-1024x635.jpg")
nobita1 = Image("https://m.media-amazon.com/images/M/MV5BM2E5MDIwNjktOWQwYS00ODg5LTlhN2MtNzU1OTc3MDA0NjNmXkEyXkFqcGc@._V1_FMjpg_UX1000_.jpg")

response = ip(images=[doraemon1, nobita1])

In [16]:
response

Prediction(
    description='The first image shows the character Doraemon, a blue robotic cat from a Japanese manga and anime series. He is depicted smiling and waving with one hand, wearing his signature bell around his neck. The word "DORAEMON" is written in large, bold letters below him. The background is a light blue color.\n\nThe second image shows a close-up of a boy wearing glasses and a yellow shirt. He is smiling widely, showing his teeth. The background appears to be a rooftop with greenery.'
)

In [17]:
for h in lm.history[-1].request:
    print(h.role, h.content)
    print("="*10)

system Your output fields are:
1. description (<class 'str'>): description of images

All interactions will be structured in the following way, with the appropriate values filled in.

<||description||>
{description}

<||completed||>
In adhering to this structure, your objective is: 
Describe images
user Respond with the corresponding output fields, starting with the field: `<||description||>` and then ending with the marker for `<||completed||>`.
